# Die helmholzgleichung:
$\Delta u - \omega^2 u = f$
### und die Schwache Formulierung:
$\int_{\Omega}\nabla u \nabla v - \omega^2\int_{\Omega}u v = \int_{\Omega}fv$ $v \in V$
### löse die obrige Gleichung mit paar $\omega_i$ und speichere diese $u_i$ in eine Matrix 

In [ ]:
#import netgen.gui
from ngsolve import *
import numpy as np
from math import pi
import netgen.geom2d as geom2dim
import matplotlib.pyplot as plt
import scipy.sparse as sp

In [ ]:
def GreatGeometry(a,shift,h):
    geo = geom2dim.SplineGeometry()
    points = [(0,0),(a,0),(a,a),(0,a)]
    p1,p2,p3,p4 = [geo.AppendPoint(*pnt) for pnt in points]
    curves = [['line',p1,p2],['line',p2,p3],['line',p3,p4],['line',p4,p1]]
    [geo.Append(c) for c in curves]

    mesh = Mesh(geo.GenerateMesh(maxh = h))


    piha = a/2
    source = exp(-50**2*((x-piha-shift)*(x-piha-shift)+(y-piha-shift)*(y-piha-shift)))
    Draw(source,mesh,name = "pulse")
    return mesh,source

In [ ]:
def plot_Reso(x,y):#,xlab,ylab,tit):
    plt.grid()
    plt.plot(x,y)
    plt.xlabel("omega")
    plt.ylabel("Euklidic-Norm")
    #plt.savefig('results/resonanz_20.png')

In [ ]:
def Redusebasis(erg,Mh,Kh,Fh):    
    rows,cols,vals = Mh.COO()
    Mh = sp.csr_matrix((vals,(rows,cols)))
    rows,cols,vals = Kh.COO()
    Kh = sp.csr_matrix((vals,(rows,cols)))                     
    V, r = np.linalg.qr(erg)
    V = erg.dot(np.linalg.inv(r))
    
    MN = np.transpose(V).dot(Mh.dot(V))
    KN = np.transpose(V).dot(Kh.dot(V))
    FN = np.transpose(V).dot(Fh)
    print("Check: ",np.vdot(V[:,1],V[:,2]))
    #print("Check: ",V[:,1].vdot(V[:,1])) # nachschauen ob complex 
    print(type(V))
    return MN,KN,FN,V

In [ ]:
def calcReduseRes(V,iteration,interval,fes,Mh,Kh,Fh,uN,o,KN,MN,FN):
    ured = GridFunction(fes)
    F = Mh.CreateColVector()
    F.FV().NumPy()[:] = Fh[:]
    #print("Fh: ",Fh)
    #print("F: ",F)
    factor = interval / iteration
    omegaN = []
    res = []
    res_snap = []
    for i in range(iteration):
        omegaN.append(factor*i)
        with TaskManager():
            ured.vec.FV().NumPy()[:] = V.dot(uN[:,i])[:]
            
            ngs_temp = Mh.CreateColVector()
            ngs_temp.data = Kh*ured.vec - omegaN[i]**2*Mh*ured.vec - F

            ngs_temp2 = Kh.CreateColVector()
            ngs_temp2.data = Projector(fes.FreeDofs(), True)*ngs_temp
            res.append(Norm(ngs_temp2))
    plt.figure()
    plot_Reso(omegaN,res)
    j = 0
    for fre in o:
        with TaskManager():
            uN[:,j] = np.linalg.solve(KN-fre**2*MN, FN)
            ured.vec.FV().NumPy()[:] = V.dot(uN[:,i])[:]
            ngs_temp = Mh.CreateColVector()
            ngs_temp.data = Kh*ured.vec - omegaN[i]**2*Mh*ured.vec - F

            ngs_temp2 = Kh.CreateColVector()
            ngs_temp2.data = Projector(fes.FreeDofs(), True)*ngs_temp
            res_snap.append(Norm(ngs_temp2))
            j = j+1
    plt.figure()
    plot_Reso(o,res_snap)

In [ ]:
def calcRedusesol(MN,KN,FN,iteration,interval,maxind):
    factor = interval / iteration
    omegaN = []
    norm =[]
    uN = np.zeros([maxind,iteration])
    for i in range(iteration):
        omegaN.append(factor*i)
        uN[:,i] = np.linalg.solve(KN-omegaN[i]**2*MN, FN)
        norm.append(sqrt(uN[:,i].dot(uN[:,i])))
    plot_Reso(omegaN,norm)
    return uN

In [ ]:
def Snapshot(name,maxind,a,shift,h,Print,points):
    mesh,source = GreatGeometry(a,shift,h)
    if name == "Dirichlet":
        fes = H1(mesh, order=5,dirichlet=".*")
        gfu = GridFunction(fes,name = "u_Dir")
        erg = np.zeros([len(gfu.vec),maxind])
        Fh = np.zeros([len(gfu.vec),maxind])
    if name == "Neumann":
        fes = H1(mesh, order=5)
        gfu = GridFunction(fes,name = "u_Neu")
        erg = np.zeros([len(gfu.vec),maxind])
        Fh = np.zeros([len(gfu.vec),maxind])
    if name == "Robin":
        fes = H1(mesh, order=5,complex = True)
        gfu = GridFunction(fes,name = "u_Rob")
        erg = np.zeros([len(gfu.vec),maxind],dtype=complex)
        Fh = np.zeros([len(gfu.vec),maxind],dtype=complex)
       
    print("Modus: ",name) 
    
    u = fes.TrialFunction()
    v = fes.TestFunction()
    
    A = BilinearForm(fes)
    K = BilinearForm(fes)
    M = BilinearForm(fes)
    omega = Parameter(1)
    if name == "Robin":
        R = BilinearForm(fes)
        A += (grad(u)*grad(v)- omega**2*u*v)*dx -1j*omega*u*v*ds
        R += -1j*u*v*ds
        R.Assemble()
    else:
        A += (grad(u)*grad(v)- omega**2*u*v)*dx        
    
    F = LinearForm(fes)
    

    K += grad(u)*grad(v)*dx
    M += u*v*dx
    
    F += source*v*dx 
    o = []
    norm = []
    j = 0
    for i in range(maxind):
        o.append(i)
    print("Start calculation the snapshots")
    for i in o:  
        with TaskManager():
            omega.Set(i) 
            A.Assemble()
            K.Assemble()
            M.Assemble()
            F.Assemble()  
            
            gfu.vec.data = A.mat.Inverse(fes.FreeDofs()) * F.vec
            erg[:,j] = gfu.vec
            Fh[:,j] = F.vec
            norm.append(sqrt (Integrate ((gfu)*(gfu),mesh)))
            if Print:
                print("Number of iter: ",j,"omega: ",i,"L2-Norm:",sqrt (Integrate ((gfu)*(gfu),mesh)))
            j = j+1
    print("plot the Resonanz: ")
    #plot_Reso(o,norm)
    print("calculates the ReducedBasis: ")
    MN,KN,FN,V = Redusebasis(erg,M.mat,K.mat,Fh[:,0])

    print("calculates the ReducedSolution: ")
    #uN = calcRedusesol(MN,KN,FN,points,20,20)
    
    print("calculates the ReducedResidual: ")
    #res = calcReduseRes(V,points,20,fes,M.mat,K.mat,Fh[:,0],uN,o,KN,MN,FN)

    #print(Fh[:,0])
    #print(Fh[:,1])
    #print(Fh[:,2])

In [ ]:
# name, maxiter , recksize, shift, h, Print,points
Snapshot("Robin",20,1,0,0.1,True,int(1e3))

In [ ]:
r

In [ ]:
o